# Desarrollo de machine learning accidentes de bicicleta

#### Cargamos las librerias necesarias

In [81]:
import numpy as np
import pandas as pd

# import the KNNimputer class
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from feature_engine.imputation import CategoricalImputer

ModuleNotFoundError: No module named 'feature_engine'

#### Primero vamos a obtener los datos de datos.madrid.es, en primer lugar obtendremos todos los datos disponibles desde 2020 inicio de la pandemia hasta hoy 2023

#### Estos serán nuestros ficheros de estudio: 
* AccidentesBicicletas_2020.xlsx 
* AccidentesBicicletas_2021.xlsx 
* AccidentesBicicletas_2022.xlsx 
* AccidentesBicicletas_2023.xlsx


##Carga de datos y preprocesado

In [40]:
# cargamos los ficheros xlsx
bicisXcel20 = pd.read_excel('data/AccidentesBicicletas_2020.xlsx')
bicisXcel21 = pd.read_excel('data/AccidentesBicicletas_2021.xlsx')
bicisXcel22 = pd.read_excel('data/AccidentesBicicletas_2022.xlsx')
bicisXcel23 = pd.read_excel('data/AccidentesBicicletas_2023.xlsx')

# Leemos los valores del fichero y los cargamos en un dataframe
bicis20 = pd.DataFrame(bicisXcel20)
bicis21 = pd.DataFrame(bicisXcel21)
bicis22 = pd.DataFrame(bicisXcel22)
bicis23 = pd.DataFrame(bicisXcel23)

dfBicis = pd.concat([bicis20, bicis21, bicis22, bicis23])
dfBicis

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehículo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,tipo_vehiculo
0,2020S000044,2020-01-02,07:30:00,CALL. ALBASANZ / CALL. RUFINO GONZALEZ,23,20.0,SAN BLAS-CANILLEJAS,Colisión frontal,Despejado,Bicicleta,Conductor,De 40 a 44 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,4.468640e+08,4.476526e+09,N,NaN,NaN
1,2020S000151,2020-01-04,13:45:00,"CALL. ORENSE, 12",12,6.0,TETUÁN,Caída,Despejado,Bicicleta,Conductor,De 21 a 24 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,4.410475e+08,4.477835e+09,N,NaN,NaN
2,2020S000153,2020-01-04,12:10:00,"CALL. NUESTRA SEÑORA DE VALVERDE, 209",209,8.0,FUENCARRAL-EL PARDO,Colisión múltiple,Despejado,Bicicleta,Conductor,De 10 a 14 años,Hombre,1.0,Atención en urgencias sin posterior ingreso,4.419754e+08,4.484542e+09,N,NaN,NaN
3,2020S000153,2020-01-04,12:10:00,"CALL. NUESTRA SEÑORA DE VALVERDE, 209",209,8.0,FUENCARRAL-EL PARDO,Colisión múltiple,Despejado,Bicicleta,Conductor,De 45 a 49 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,4.419754e+08,4.484542e+09,N,NaN,NaN
4,2020S000212,2020-01-04,13:30:00,"M-605, KM 1,600 SOMONTES (CAMINO)",20,8.0,FUENCARRAL-EL PARDO,Caída,Despejado,Bicicleta,Conductor,De 35 a 39 años,Hombre,6.0,Asistencia sanitaria inmediata en centro de sa...,4.391659e+08,4.484558e+09,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12183,2023S013443,2023-03-31,13:30:00,CALL. MENDEZ ALVARO / CALL. RETAMA,93,13.0,PUENTE DE VALLECAS,Alcance,Despejado,NaN,Conductor,De 25 a 29 años,Hombre,NaN,NaN,4.425613e+05,4.471591e+06,N,NaN,Motocicleta hasta 125cc
12184,2023S013443,2023-03-31,13:30:00,CALL. MENDEZ ALVARO / CALL. RETAMA,93,13.0,PUENTE DE VALLECAS,Alcance,Despejado,NaN,Conductor,De 40 a 44 años,Hombre,NaN,NaN,4.425613e+05,4.471591e+06,N,NaN,Turismo
12185,2023S013443,2023-03-31,13:30:00,CALL. MENDEZ ALVARO / CALL. RETAMA,93,13.0,PUENTE DE VALLECAS,Alcance,Despejado,NaN,Pasajero,De 40 a 44 años,Mujer,NaN,NaN,4.425613e+05,4.471591e+06,N,NaN,Turismo
12186,2023S014588,2023-03-17,11:50:00,PASEO. CASTELLANA / CUZCO,162,5.0,CHAMARTÍN,Alcance,Despejado,NaN,Conductor,Desconocido,Desconocido,NaN,NaN,4.414998e+05,4.478870e+06,N,NaN,NaN


In [41]:
# dimensiones
dfBicis.shape

(14840, 20)

#### Son un total de 14840 observaciones agrupadas en 21 categorías: id, num_expediente,	fecha,	hora,	localizacion,	numero,cod_distrito,	distrito,	tipo_accidente,	estado_meteorológico,	tipo_vehículo, 	rango_edad,	sexo,	cod_lesividad,	tipo_lesividad,	coordenada_x_utm,	coordenada_y_utm,	positiva_alcohol,	positiva_droga,	tipo_vehiculo,	lesividad

In [42]:
# chequeamos los valores nulos de nuestro dataSet
dfBicis.isnull().sum()

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                      1
cod_distrito                1
distrito                    1
tipo_accidente              0
estado_meteorológico     1491
tipo_vehículo           13066
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad            6278
lesividad                6278
coordenada_x_utm            2
coordenada_y_utm            2
positiva_alcohol           56
positiva_droga          14790
tipo_vehiculo            1871
dtype: int64

In [43]:
dfBicis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14840 entries, 0 to 12187
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   num_expediente        14840 non-null  object        
 1   fecha                 14840 non-null  datetime64[ns]
 2   hora                  14840 non-null  object        
 3   localizacion          14840 non-null  object        
 4   numero                14839 non-null  object        
 5   cod_distrito          14839 non-null  float64       
 6   distrito              14839 non-null  object        
 7   tipo_accidente        14840 non-null  object        
 8   estado_meteorológico  13349 non-null  object        
 9   tipo_vehículo         1774 non-null   object        
 10  tipo_persona          14840 non-null  object        
 11  rango_edad            14840 non-null  object        
 12  sexo                  14840 non-null  object        
 13  cod_lesividad   

Eliminación de columnas que no se usarán en el análisis:
* Nº expediente de la Policia no es necesario
* Localizacion no es necesario saber la calle exacta del accidente, ya se agruparán los resultados por DISTRITO
* Nº de calle al ser demasiado específico para este estudio
* Tipo Vehiculo o tipo vehículo en este estudio siempre es BICICLETA
* TIPO PERSONA no aporta información relevante para el estudio
* cod_distrito no es necesario para nuestro analisis, ya que tenemos el nombre de distrito
* cod_lesividad no es necesario para nuestro analisis, ya que tenemos la descripcion de lesividad

In [44]:
# Realizamos un backup del dataFrame de bicis
dfBicisBackup = dfBicis

In [45]:
dfBicis = dfBicis.drop(['num_expediente', 'localizacion', 'numero', 'tipo_vehículo', 'tipo_vehiculo', 'cod_distrito', 'cod_lesividad'], axis=1)

In [46]:
dfBicis

,fecha,hora,distrito,tipo_accidente,estado_meteorológico,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2020-01-02,07:30:00,SAN BLAS-CANILLEJAS,Colisión frontal,Despejado,Conductor,De 40 a 44 años,Hombre,Asistencia sanitaria sólo en el lugar del acci...,4.468640e+08,4.476526e+09,N,NaN
1,2020-01-04,13:45:00,TETUÁN,Caída,Despejado,Conductor,De 21 a 24 años,Hombre,Asistencia sanitaria sólo en el lugar del acci...,4.410475e+08,4.477835e+09,N,NaN
2,2020-01-04,12:10:00,FUENCARRAL-EL PARDO,Colisión múltiple,Despejado,Conductor,De 10 a 14 años,Hombre,Atención en urgencias sin posterior ingreso,4.419754e+08,4.484542e+09,N,NaN
3,2020-01-04,12:10:00,FUENCARRAL-EL PARDO,Colisión múltiple,Despejado,Conductor,De 45 a 49 años,Hombre,Ingreso inferior o igual a 24 horas,4.419754e+08,4.484542e+09,N,NaN
4,2020-01-04,13:30:00,FUENCARRAL-EL PARDO,Caída,Despejado,Conductor,De 35 a 39 años,Hombre,Asistencia sanitaria inmediata en centro de sa...,4.391659e+08,4.484558e+09,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12183,2023-03-31,13:30:00,PUENTE DE VALLECAS,Alcance,Despejado,Conductor,De 25 a 29 años,Hombre,NaN,4.425613e+05,4.471591e+06,N,NaN
12184,2023-03-31,13:30:00,PUENTE DE VALLECAS,Alcance,Despejado,Conductor,De 40 a 44 años,Hombre,NaN,4.425613e+05,4.471591e+06,N,NaN
12185,2023-03-31,13:30:00,PUENTE DE VALLECAS,Alcance,Despejado,Pasajero,De 40 a 44 años,Mujer,NaN,4.425613e+05,4.471591e+06,N,NaN
12186,2023-03-17,11:50:00,CHAMARTÍN,Alcance,Despejado,Conductor,Desconocido,Desconocido,NaN,4.414998e+05,4.478870e+06,N,NaN


In [47]:
dfBicis.isnull().sum()

fecha                       0
hora                        0
distrito                    1
tipo_accidente              0
estado_meteorológico     1491
tipo_persona                0
rango_edad                  0
sexo                        0
lesividad                6278
coordenada_x_utm            2
coordenada_y_utm            2
positiva_alcohol           56
positiva_droga          14790
dtype: int64

Vamos a tratar los valores nulos, para las variables null:
* distrito solo tiene uno eliminamos esa fila
* coordenada_y_utm y coordenada_x_utm solo hay dos, las eliminamos
* positiva_alcohol solo tiene 56 las eliminamos
* positiva_droga la mayoría son null suponemos porque es negativo, por lo que 1 es positivo y 0 negativo
* lesividad, dado que hay un numero elevado vamos a realizar tareas de asignación de valores.
* estado_meteorologico, hay un gran número vamos a realizar tareas de asignación de valores.

In [48]:
dfBicis = dfBicis.dropna(subset = ['distrito', 'coordenada_y_utm', 'coordenada_x_utm', 'positiva_alcohol'])

In [49]:
dfBicis.isnull().sum()

fecha                       0
hora                        0
distrito                    0
tipo_accidente              0
estado_meteorológico     1482
tipo_persona                0
rango_edad                  0
sexo                        0
lesividad                6221
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol            0
positiva_droga          14731
dtype: int64

In [50]:
dfBicis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14781 entries, 0 to 12187
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   fecha                 14781 non-null  datetime64[ns]
 1   hora                  14781 non-null  object        
 2   distrito              14781 non-null  object        
 3   tipo_accidente        14781 non-null  object        
 4   estado_meteorológico  13299 non-null  object        
 5   tipo_persona          14781 non-null  object        
 6   rango_edad            14781 non-null  object        
 7   sexo                  14781 non-null  object        
 8   lesividad             8560 non-null   object        
 9   coordenada_x_utm      14781 non-null  float64       
 10  coordenada_y_utm      14781 non-null  float64       
 11  positiva_alcohol      14781 non-null  object        
 12  positiva_droga        50 non-null     float64       
dtypes: datetime64[ns

In [52]:
dfBicis['positiva_droga'] = dfBicis['positiva_droga'].fillna(0)

C:\Users\javi4\AppData\Local\Temp/ipykernel_6712/3753712652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfBicis['positiva_droga'] = dfBicis['positiva_droga'].fillna(0)


In [54]:
dfBicis.isnull().sum()

fecha                      0
hora                       0
distrito                   0
tipo_accidente             0
estado_meteorológico    1482
tipo_persona               0
rango_edad                 0
sexo                       0
lesividad               6221
coordenada_x_utm           0
coordenada_y_utm           0
positiva_alcohol           0
positiva_droga             0
dtype: int64

In [ ]:
# Para imputar valores primero hay que transformar las variables a categorical

In [61]:
dfBicis['estado_meteorológico'] = dfBicis.estado_meteorológico.astype('category')

C:\Users\javi4\AppData\Local\Temp/ipykernel_6712/16681083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfBicis['estado_meteorológico'] = dfBicis.estado_meteorológico.astype('category')


In [62]:
dfBicis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14781 entries, 0 to 12187
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   fecha                 14781 non-null  datetime64[ns]
 1   hora                  14781 non-null  object        
 2   distrito              14781 non-null  object        
 3   tipo_accidente        14781 non-null  object        
 4   estado_meteorológico  13299 non-null  category      
 5   tipo_persona          14781 non-null  object        
 6   rango_edad            14781 non-null  object        
 7   sexo                  14781 non-null  object        
 8   lesividad             8560 non-null   object        
 9   coordenada_x_utm      14781 non-null  float64       
 10  coordenada_y_utm      14781 non-null  float64       
 11  positiva_alcohol      14781 non-null  object        
 12  positiva_droga        14781 non-null  float64       
dtypes: category(1), 

## Imputación de valores perdidos

La librería scikit-learn nos proporciona la clase KNNImputer para hacer uso de este modelo en la imputación de missing values. Esta clase usa por defecto la distancia euclidiana, pero podemos elegir la que prefiramos modificando el parámetro metric. Asimismo, también podemos elegir el número de vecinos con el argumento n_neighbors y en esta ocasión usaremos 5. Podeís ver en el siguiente código como utilizamos este método para imputar los valores perdidos en la variable normalized-losses:

In [80]:
# configurar la imputación
imputer = CategoricalImputer(
    imputation_method='frequent',
    variables=['estado_meteorológico','lesividad'],)
# ajustar la imputación
imputer.fit(dfBicis[['estado_meteorológico','lesividad']])
dfBicis['estado_meteorológico','lesividad'] = imputer.transform(dfBicis[['estado_meteorológico','lesividad']])

NameError: name 'CategoricalImputer' is not defined

In [67]:
# Construimos el modelo
imputer = KNNImputer(n_neighbors=5, weights="uniform")

# Ajustamos el modelo e imputamos los missing values
imputer.fit(dfBicis[['estado_meteorológico','lesividad']])
dfBicis['estado_meteorológico','lesividad'] = imputer.fit_transform(dfBicis[['estado_meteorológico','lesividad']]).ravel()

ValueError: could not convert string to float: 'Despejado'

In [74]:
dfBicis.isnull().sum()

fecha                       0
hora                        0
distrito                    0
tipo_accidente              0
estado_meteorológico    14781
tipo_persona                0
rango_edad                  0
sexo                        0
lesividad                6221
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol            0
positiva_droga              0
dtype: int64

El problema es que la mayoría de los modelos de machine learning en Python, entre ellos las redes neuronales, solo pueden leer valores numéricos puesto que que hacen uso de diversas operaciones matemáticas. Por esto mismo, en estos casos es importante convertir este tipo de variables en numéricas para que los modelos puedan utilizarlas.
Comenzamos con esta popular técnica de codificación que consiste simplemente en remplazar cada valor de la variable con un número entero distinto. Es por ello que la codificación ordinal resulta útil cuando tenemos datos ordinales, es decir, aquellos en los que se puede establecer un orden entre sus categóricas. En consecuencia, esta técnica nos ayudará a representar esta relación de orden de las distintas categorías.

In [68]:
# Para la variable estado_meteorológico tenemos los siguientes valores: Despejado, Nublado, Lluvia débil, Se desconoce, Lluvia intensa.
dfBicis.estado_meteorológico.value_counts()

Despejado         11923
Nublado             731
Lluvia débil        514
Se desconoce         72
LLuvia intensa       59
Name: estado_meteorológico, dtype: int64

In [70]:
# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[["Despejado", "Nublado", 
                                      "Lluvia débil", "LLuvia intensa", "Se desconoce"]])

# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(dfBicis[["estado_meteorológico"]])
dfBicis["estado_meteorológico-encoded"] = encoder.transform(dfBicis[["estado_meteorológico"]])

ValueError: Input contains NaN

In [82]:
dfBicis.lesividad.value_counts()

Sin asistencia sanitaria                                     3822
Asistencia sanitaria sólo en el lugar del accidente          2281
Ingreso inferior o igual a 24 horas                           848
Atención en urgencias sin posterior ingreso                   644
Asistencia sanitaria inmediata en centro de salud o mutua     484
Asistencia sanitaria ambulatoria con posterioridad            246
Ingreso superior a 24 horas                                   226
Fallecido 24 horas                                              8
Se desconoce                                                    1
Name: lesividad, dtype: int64